In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import dask.dataframe as dd
import pandas as pd
import plotly.express as px
import xarray as xr

from weather_weaver.utils import load_world_countries
from weather_weaver.inputs.ecmwf.constants import ENTSO_E_ISO3_LIST

from weather_weaver.inputs.ecmwf.open_data import (
    ECMWFOpenDataFetcher,
    ECMWFOpenDataRequest,
    EMCWFOpenDataProcessor,
    GeoFilterModel,
    StreamType,
    RunTime,
    RequestType,
)

pd.set_option("display.max_columns", 500)

In [3]:
world = load_world_countries()

In [4]:
geo_filter_df = world[world["country_iso3"].isin(ENTSO_E_ISO3_LIST)].copy()

# define the geo filter model
geo_filter = GeoFilterModel(filter_df=geo_filter_df, method="within")

In [7]:
run_date = dt.date(2024, 1, 1)
run_time = RunTime.H00
stream =  StreamType.OPER
request_type = RequestType.FORECAST

data_request = ECMWFOpenDataRequest(
    run_date=run_date,
    run_time=run_time,
    stream=stream,
    request_type=request_type,
)

In [8]:
fetcher = ECMWFOpenDataFetcher()

2024-01-04 17:11:19 [debug    ] Init fetcher                   data_dir=PosixPath('/Users/badrbenmbarek/Documents/work/git/weather-weaver/data/ecmwf/raw') data_source=azure source=ECMWF


In [9]:
out_path = fetcher.download_raw_files(data_request)

2024-01-04 17:11:19 [debug    ] Download raw files skipped.    data_source=azure destination_path=PosixPath('/Users/badrbenmbarek/Documents/work/git/weather-weaver/data/ecmwf/raw/oper/20240101_00z_0-90_oper_fc.grib2') fetcher=ECMWFOpenDataFetcher request=ECMWFOpenDataRequest(run_date=datetime.date(2024, 1, 1), run_time=<RunTime.H00: 0>, stream=<StreamType.OPER: 'oper'>, request_type=<RequestType.FORECAST: 'fc'>, nwp_parameters=['2t', 'tp', '10u', '10v'], forecast_steps=[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90])


In [10]:
processor = EMCWFOpenDataProcessor()

In [11]:
ddf = processor.transform(raw_path=out_path, request=data_request, filter_model=geo_filter)

In [ ]:
ddf.to_parquet(PROCESSED_STREAM_PATH / f"{file_name}.parquet")

In [ ]:
# read back saved parquet file

In [ ]:
test_df = dd.read_parquet(RAW_STREAM_PATH / f"{file_name}.parquet")

In [ ]:
tt = test_df.groupby(["value_datetime", "NAME", "variable"]).agg({"value": "mean"})

In [ ]:
ttt = tt.compute()

In [ ]:
fig = px.line(ttt.reset_index(), x="value_datetime", y="value", color="NAME", facet_col="variable")
fig.show()

In [ ]:
path = "/Users/badrbenmbarek/Downloads/ECMWF_HRES_06Z_20190101.nc"

In [ ]:
dds = xr.open_dataset(path)
dds.

In [ ]:
area: str = "uk"
hours: int = 48
param_group: str = "basic"

start = dt.date(2023, 12, 26)
end = dt.date(2023, 12, 27)

In [ ]:
match param_group:
    case "basic":
        parameters = ["167.128/169.128"]  # 2 Metre Temperature, Dswrf
    case _:
        parameters = list(PARAMETER_ECMWFCODE_MAP.keys())

In [ ]:
all_init_times: list[dt.datetime] = [
    pdt.to_pydatetime()
    for pdt in pd.date_range(
        start=start,
        end=end + dt.timedelta(days=1),
        inclusive="left",
        freq="H",
        tz=dt.UTC,
    ).tolist()
    if pdt.hour in [0, 6, 12, 18]
]

In [ ]:
for init_time in all_init_times:
    mars_request = buildMarsRequest(parameters=parameters, hours=hours, area=area, it=init_time, list_only=True, target="test.txt")
    break

In [ ]:
print(mars_request)

In [ ]:
server.execute(mars_request, target="test.txt")